<a href="https://colab.research.google.com/github/hits-sdo/hits-sdo-similaritysearch/blob/byol-training_pipeline/search_byol/byol_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to Intialize HITS-SDO self-similarity search environment
- Run all cells to initalize environment, and restart runtime if prompted to use updated versions. You will need to rerun the cells again to ensure that all dependencies have been installed.

# Download and Unzip Data

In [1]:
# Download Data
!gdown 15C5spf1la7L09kvWXll2qt67Ec0rwLsY

Downloading...
From: https://drive.google.com/uc?id=15C5spf1la7L09kvWXll2qt67Ec0rwLsY
To: /content/aia_171_color_1perMonth.tar.gz
100% 146M/146M [00:00<00:00, 179MB/s]


In [2]:
# Unzip file
!tar -zxf aia_171_color_1perMonth.tar.gz

In [3]:
# Print some files to see that they exist
!du aia_171_color_1perMonth/. -l -h

40K	aia_171_color_1perMonth/./20211021_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20211021_000036_aia.lev1_euv_12s_4k/tiles
3.2M	aia_171_color_1perMonth/./20211021_000036_aia.lev1_euv_12s_4k
40K	aia_171_color_1perMonth/./20171010_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20171010_000036_aia.lev1_euv_12s_4k/tiles
3.2M	aia_171_color_1perMonth/./20171010_000036_aia.lev1_euv_12s_4k
40K	aia_171_color_1perMonth/./20160617_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20160617_000036_aia.lev1_euv_12s_4k/tiles
3.2M	aia_171_color_1perMonth/./20160617_000036_aia.lev1_euv_12s_4k
40K	aia_171_color_1perMonth/./20191117_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20191117_000036_aia.lev1_euv_12s_4k/tiles
3.2M	aia_171_color_1perMonth/./20191117_000036_aia.lev1_euv_12s_4k
40K	aia_171_color_1perMonth/./20210919_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./2021

# Clone repository

In [1]:
# Clone the repository from GitHub
!git clone https://github.com/hits-sdo/hits-sdo-similaritysearch

fatal: destination path 'hits-sdo-similaritysearch' already exists and is not an empty directory.


In [2]:
%cd hits-sdo-similaritysearch/

/content/hits-sdo-similaritysearch


# Switch to Desired Branch

In [3]:
# Switch to the desired branch with requirements.txt
!git checkout byol-training_pipeline

Already on 'byol-training_pipeline'
Your branch is up to date with 'origin/byol-training_pipeline'.


In [4]:
# Confirm that branch is up to date
!git log --oneline

f7937e2 (HEAD -> byol-training_pipeline, origin/byol-training_pipeline) added default datatype and made it single float
570ff76 Add tqdm bars to training notebook
f666699 Add dataset stride to work with even smaller sets
899150b Merge remote-tracking branch 'origin/byol-augmentation' into byol-training_pipeline
66b4624 (origin/byol-augmentation) test and ensure that dimensions are chanels first
570365f Update data loader in byol_train
cb8037b fix opencv pip repository in requirements
0db1aa6 add a version requirement to opencv
0dd5485 Merge branch 'byol-training_pipeline' of https://github.com/hits-sdo/hits-sdo-similaritysearch into byol-training_pipeline
4183c2c Add opencv to requirements
7b7e31d Add link to colab
e99626e Re-commit training notebook
464356e Initalize BYOL training notebook
f6a7bcf Rename database class and add augmentations
6b23428 (origin/byol-loader) Initialize database and its test suite
d83a4b3 Add npy files to gitignore
acdcd3c Merge pull request #1 from hits-sdo

# Install all necesary packages into environment

In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hits-sdo/hits-sdo-packager.git (to revision pip_nodata) to /tmp/pip-req-build-3ci3dk6a
  Running command git clone --filter=blob:none --quiet https://github.com/hits-sdo/hits-sdo-packager.git /tmp/pip-req-build-3ci3dk6a
  Running command git checkout -b pip_nodata --track origin/pip_nodata
  Switched to a new branch 'pip_nodata'
  Branch 'pip_nodata' set up to track remote branch 'pip_nodata' from 'origin'.
  Resolved https://github.com/hits-sdo/hits-sdo-packager.git to commit 3a54caba2ae6bf7caf4eabaf580a6ffda1b3bbda
  Preparing metadata (setup.py) ... done


##  Load Modules

In [6]:
import copy
import numpy as np
from tqdm.autonotebook import tqdm

import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLPredictionHead, BYOLProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.utils.scheduler import cosine_schedule

from search_byol.database import SDOTilesDataset

<ipython-input-6-3671dbb1c67e>:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Define BYOL Model

In [7]:
class BYOL(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = BYOLProjectionHead(512, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z

## Initialize Module

In [8]:
resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BYOL(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BYOL(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

## Initialize Dataloader

In [9]:
data_path = '/content/aia_171_color_1perMonth'
dataset = SDOTilesDataset(data_path=data_path, double_augmentation=False, data_stride=10)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=2,
)

## Run Training Loop 

In [27]:
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

epochs = 5
avg_loss = np.nan
print("Starting Training")
epoch = 0
epochs_bar = tqdm(range(epochs), dynamic_ncols=True, desc=f'Epoch {epoch:>02} - Av. loss: {avg_loss:.5f}')
for epoch in epochs_bar:
    total_loss = 0
    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    batches_bar = tqdm(dataloader, dynamic_ncols=True, leave=True, desc=f'Batches - Av. loss: {avg_loss:.5f}')
    for (x0, x1) in batches_bar:
        update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
        update_momentum(
            model.projection_head, model.projection_head_momentum, m=momentum_val
        )
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        avg_loss = total_loss / len(dataloader)
        batches_bar.set_description(f'Epoch {epoch:>02} - Av. loss: {avg_loss:.5f} - Batches')
        batches_bar.refresh()        

    epochs_bar.set_description(f'Epoch {epoch:>02} - Av. loss: {avg_loss:.5f}')
    epochs_bar.refresh()      

Starting Training


Epoch 00 - Av. loss: nan:   0%|          | 0/5 [00:00<?, ?it/s]

Batches - Av. loss: nan:   0%|          | 0/41 [00:00<?, ?it/s]

Batches - Av. loss: -0.97875:   0%|          | 0/41 [00:00<?, ?it/s]

Batches - Av. loss: -0.97981:   0%|          | 0/41 [00:00<?, ?it/s]

Batches - Av. loss: -0.98113:   0%|          | 0/41 [00:00<?, ?it/s]

Batches - Av. loss: -0.98167:   0%|          | 0/41 [00:00<?, ?it/s]